Load all necessary imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import cv2
import matplotlib.pyplot as plt
import h5py

# Configure Keras to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

2024-06-11 17:22:56.594626: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 17:22:56.628682: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 17:22:56.628710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 17:22:56.629641: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-11 17:22:56.635170: I tensorflow/core/platform/cpu_feature_guar

Data Loading

In [3]:
with h5py.File('/home/m3-learning/Documents/Research Data/Simple Object Detection/Data/TrainingData5zeroes midpoints.h5', 'r') as hdf:
    ls = list(hdf.keys())
    images = hdf.get('images')
    boxes = hdf.get('boxes')
    images = np.array(images)
    boxes = np.array(boxes)

Data Normalization

In [4]:
image_normalized = (images+1e-9)/9.26 # Normalize images and add noise


# Normalize bounding boxes
normalized_boxes = boxes / [1,64,64,64,64]

Model Train


In [5]:
from tensorflow.keras import layers, models



input_shape = (64, 64, 1)
num_classes = 280
num_coordinates = 4

images_np = image_normalized
#images_np = tf.expand_dims(images_np, axis=-1)
probabilities = np.array(normalized_boxes[:, :,:-4])
probabilities = tf.expand_dims(probabilities, axis=1)
boxes_np = np.array(normalized_boxes[:, :, 1:])
boxes_np = tf.expand_dims(boxes_np, axis=1)

dataset = tf.data.Dataset.from_tensor_slices((images_np,{'x_prob_reshape':probabilities,'x_boxes_reshape':boxes_np}))
dataset = dataset.shuffle(buffer_size=10000,reshuffle_each_iteration=True)
dataset = dataset.batch(128)

# # Assuming dataset has 10,000 examples and we want an 80/20 split
# total_items = 10000
# train_size = int(total_items * 0.6)
# val_size = total_items - train_size

# # Splitting the dataset
# train_dataset = dataset.take(train_size)
# val_dataset = dataset.skip(train_size)


# train_dataset = train_dataset.shuffle(buffer_size=train_size,reshuffle_each_iteration=True)
# train_dataset = train_dataset.batch(64)
# val_dataset = val_dataset.batch(64) 



x_input = layers.Input(shape=input_shape)

x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
x = layers.MaxPool2D()(x)
x = layers.BatchNormalization()(x) 

x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
 

x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)


x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)


x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)


x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)


x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)


x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)


x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)


x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.BatchNormalization()(x) # size: 8x8x


x = layers.Flatten()(x)
# Bounding box output
x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)

x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)


model = tf.keras.models.Model(x_input, [x_prob_reshape, x_boxes_reshape])


optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)  # Consider adjusting based on performance





model.compile(optimizer= optimizer, loss= {'x_prob_reshape': tf.keras.losses.BinaryCrossentropy(), 'x_boxes_reshape':tf.keras.losses.MeanSquaredError()}, metrics={'x_prob_reshape': 'accuracy'} )
num_epochs = 2000


2024-06-11 17:23:51.278161: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15417 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [18]:
# model = tf.keras.models.load_model('M11overfittedmodel3variant.h5')
loadmodel = tf.keras.models.load_model("/home/m3-learning/Documents/Research Data/Simple Object Detection/Weights/Jingrui's/M11overfittedmodel3variant.keras") 

In [7]:



# model.fit(
#     dataset,
#     epochs=num_epochs 
   
# )





Epoch 1/2000


2024-03-19 13:14:58.551259: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-19 13:14:58.637889: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-19 13:14:59.669347: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-19 13:15:00.654207: I external/local_xla/xla/service/service.cc:168] XLA service 0x61af318b2d30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-19 13:15:00.654238: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-19 13:15:00.663189: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710868500.812634  192145 device_compiler.

79/79 [==============================] - 21s 93ms/step - loss: 6.5545e-05 - x_prob_reshape_loss: 1.3152e-07 - x_boxes_reshape_loss: 6.5414e-05 - x_prob_reshape_accuracy: 1.0000
Epoch 2/2000
79/79 [==============================] - 6s 70ms/step - loss: 5.1476e-05 - x_prob_reshape_loss: 1.3246e-07 - x_boxes_reshape_loss: 5.1344e-05 - x_prob_reshape_accuracy: 1.0000
Epoch 3/2000
79/79 [==============================] - 6s 71ms/step - loss: 5.0544e-05 - x_prob_reshape_loss: 1.2617e-07 - x_boxes_reshape_loss: 5.0418e-05 - x_prob_reshape_accuracy: 1.0000
Epoch 4/2000
79/79 [==============================] - 6s 70ms/step - loss: 4.8000e-05 - x_prob_reshape_loss: 1.2426e-07 - x_boxes_reshape_loss: 4.7875e-05 - x_prob_reshape_accuracy: 1.0000
Epoch 5/2000
79/79 [==============================] - 5s 67ms/step - loss: 4.8080e-05 - x_prob_reshape_loss: 1.2475e-07 - x_boxes_reshape_loss: 4.7955e-05 - x_prob_reshape_accuracy: 1.0000
Epoch 6/2000
79/79 [==============================] - 5s 67ms/step 

In [30]:
# model.save('M11overfittedmodel3variant.h5')
# 


In [31]:
# model.save('M11overfittedmodel3variant.keras')

In [7]:
boxes.shape

(10000, 280, 5)

In [19]:
dataset = tf.data.Dataset.from_tensor_slices((images_np,{'x_prob_reshape':probabilities,'x_boxes_reshape':boxes_np}))

# # Assuming dataset has 10,000 examples and we want an 80/20 split
# total_items = 10000
# train_size = int(total_items * 0.6)
# # val_size = total_items - train_size

# # Splitting the dataset
# train_dataset = dataset.take(train_size)
# # val_dataset = dataset.skip(train_size)


# # train_dataset = train_dataset.shuffle(buffer_size=train_size,reshuffle_each_iteration=True)
# train_dataset = train_dataset.batch(6000)
# inputs,targets = next(iter(train_dataset))
dataset = dataset.batch(10000)
inputs,targets = next(iter(dataset))
output = loadmodel.predict(inputs)

313/313 [==============================] - 3s 9ms/step


In [20]:
targets['x_boxes_reshape'][0]

<tf.Tensor: shape=(1, 280, 4), dtype=float64, numpy=
array([[[0.734375, 0.015625, 0.734375, 0.03125 ],
        [0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      ],
        ...,
        [0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      ]]])>

In [21]:
output[1][0]

array([[[7.26398349e-01, 2.40748804e-02, 7.40999579e-01, 2.64618546e-02],
        [9.53416806e-03, 3.95403616e-03, 9.95969400e-03, 4.04721033e-03],
        [2.11569248e-03, 2.93728849e-03, 2.35647149e-03, 3.18393460e-03],
        ...,
        [1.10354495e-05, 7.63153730e-06, 6.97237647e-06, 1.10484225e-05],
        [5.75202830e-06, 7.59269187e-06, 9.98805626e-06, 4.43214276e-06],
        [6.25059192e-06, 1.03320408e-05, 5.24672214e-06, 1.16176088e-05]]],
      dtype=float32)

In [9]:
import tensorflow as tf

r = np.random.randint(0,100)
tensor1 = tf.constant(targets['x_prob_reshape'], dtype=tf.float64)

tensor2 = tf.constant(output[0], dtype=tf.float32)


tensor2 = tf.cast(tensor2, tf.float64)


mse_loss_fn =  tf.keras.losses.BinaryCrossentropy()
mse_loss = mse_loss_fn(tensor1, tensor2)

print("MSE Loss:", mse_loss.numpy())


MSE Loss: 1.3558886642682864e-07


In [2]:
def visualize_bounding_boxes(image, probability_vector, bounding_box_coordinates, threshold=0.9):
    """
    Visualizes bounding boxes on an image based on a probability vector.

    Parameters:
    - image: A 3D tensor representing the image.
    - probability_vector: A 1D tensor representing the probabilities associated with bounding boxes.
    - bounding_box_coordinates: A 2D tensor representing bounding box coordinates.
    - threshold: Probability threshold for visualization.

    Returns:
    None (displays the image with bounding boxes).
    """
    # Convert to NumPy arrays for easier handling
    image_np = image
    prob_vector_np = probability_vector
    bbox_coordinates_np = bounding_box_coordinates
   # Denormalize image if necessary (adjust based on your normalization method)
    denormalized_image = image_np  # Modify if normalization was applied during training

    # Visualize the image
    plt.figure(figsize=(4, 4))
    plt.imshow(denormalized_image, cmap='gray')
    plt.title("Bounding Box Visualization")

    # Plot bounding boxes based on probability threshold
    for i in range(len(prob_vector_np)):
        prob = prob_vector_np[i]
        bbox = bbox_coordinates_np[i]
        if prob > threshold:
            # Denormalize bounding box coordinates if necessary
            denormalized_bbox = bbox  # Modify if normalization was applied during training
            y1, x1, y2, x2 = denormalized_bbox
            plt.plot([x1, x2, x2, x1, x1],[y1, y1, y2, y2, y1],
                     
                     color='r', linewidth=2, label='Bounding Box')

   






   
    plt.show()

t = np.random.randint(0,100)


visualize_bounding_boxes(tf.convert_to_tensor(inputs[t]), probabilities.numpy()[t].squeeze(), tf.convert_to_tensor(output[1][t,0,:,:])*[64,64,64,64]) ##myprediction
visualize_bounding_boxes(tf.convert_to_tensor(image_normalized[t]), probabilities.numpy()[t].squeeze(), tf.convert_to_tensor(boxes_np[t,0,:,:])*[64,64,64,64]) ##myprediction





NameError: name 'np' is not defined